In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
model_path = "Qwen/Qwen2.5-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    torch_dtype=torch.float16,
)
tokenizer.padding_side = 'left'

/home/rbqlsquf2/workspace/XAI_rationale-inference-LLM/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.35s/it]


In [2]:
new_special_tokens = {"additional_special_tokens": ["<|mrc|>", "<|summary|>"]}

In [5]:
tokenizer.add_special_tokens(new_special_tokens)

<bound method SpecialTokensMixin.add_special_tokens of Qwen2TokenizerFast(name_or_path='Qwen/Qwen2.5-3B-Instruct', vocab_size=151643, model_max_length=131072, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|mrc|>', '<|summary|>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151646: AddedToken("<|object_ref_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151647: AddedToken("<|object_ref_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=Tru

In [4]:
model.resize_token_embeddings(len(tokenizer))

Embedding(151667, 2048)

In [27]:
from tqdm import tqdm

def create_example(all_data):
    all_result = []
    for data in tqdm(all_data):
        data_id = data["_id"]
        Question = data["question"]
        answer = data["answer"]
        context = data["context"]
        supporting_facts = data["supporting_facts"]
        concat_supporting_sent = ""
        document_number_support_sent = ""
        support_dic = {}
        write_supporting_sent = ""
        total_sentence_dic = {}
        # 전체 sentence number 세기 위함
        total_sentence_number = 1
        document_sentence_number = 0
        support_num = []
        supporting_sentence = ""
        document = ""
        for sup_sent in supporting_facts:
            title = sup_sent[0]  # supporting fact의 제목
            set_num = sup_sent[1]
            if title not in support_dic.keys():  # 문장번호
                support_dic[title] = []
            support_dic[title].append(set_num)
        
        for index, j in enumerate(context):
            title = j[0]
            sentences = ""
            if title in support_dic:
                document_sentence_number = 0
                for sent in j[1]:
                    sentence = "[{}] {}".format(total_sentence_number, sent) + "\n"
                    sentences = sentences + sentence
                    if document_sentence_number in support_dic[title]:
                        support_num.append(total_sentence_number)
                        supporting_sentence = supporting_sentence + sentence
                    document_sentence_number += 1
                    total_sentence_number += 1

            else:
                for sent in j[1]:
                    sentence = "[{}] {}".format(total_sentence_number, sent) + "\n"
                    sentences = sentences + sentence
                    total_sentence_number += 1

            write_sent = "Document {} : {}".format(index + 1, title) + "\n" + "{}".format(sentences)
            document = document + write_sent + "\n"
        document = document.rstrip("\n")
        instruction = "<|mrc|>True\n**Question**\n{}\n<|summary|>Flase".format(Question)
        prompt = (
            "**Document**\n{}".format(document)
        )
        response = "**Answer**\n{}\n**Summary**\n".format(answer)
        # response = "**Answer**: {}\n**Supporting Sentences**: {}".format(answer, supporting_sentence)
        messages = [
            {"role": "system", "content": instruction},
            {"role": "user", "content": prompt},
            # {"role": "assistant", "content": response},
        ]
        result = {}
        result["text"] = tokenizer.apply_chat_template(messages, tokenize=False)

        messages = [
            {"role": "system", "content": instruction},
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": response},
        ]
        result["all_text"] = tokenizer.apply_chat_template(messages, tokenize=False)
        result["label"] = response
        all_result.append(result)
        supporting_sentence = ""
        # print(model_answer)
        # if len(all_result) ==1000:
        #     break
    return all_result
    

In [28]:
import json
# file_path = "data/hotpot_train_v1.1_re.json"
file_path = "data/hotpot_dev.json"
with open(file_path, "r", encoding="utf-8") as file:
    dev_data = json.load(file)

input_data = create_example(dev_data)

100%|██████████| 7405/7405 [00:00<00:00, 10944.32it/s]


In [30]:
from tqdm import tqdm
all_len = []
all_result = []
over_num = 0
for input_data_ in tqdm(input_data):
    text = input_data_["all_text"]
    count = len(tokenizer(text)["input_ids"])
    if count <= 2048:
        all_result.append(input_data_)
    else:
        over_len = count - 2048
        input_data_['text'] = input_data_['text'][:over_len]
        over_num +=1
    # all_len.append(len(tokenizer(text)["input_ids"]))
print(over_num)

100%|██████████| 7405/7405 [00:25<00:00, 292.09it/s]

1108


In [38]:
for result in all_result:
    result["label"] = "assistant\n" + result["label"]
    

In [39]:
with open("data/qwen_dev_data.json", "w", encoding="utf-8") as f:
    json.dump(all_result, f, ensure_ascii=False, indent=4)

In [14]:
from tqdm import tqdm
all_len = []
all_result = []

for data, input_data_ in tqdm(zip(dev_data, input_data)):
    text = input_data_["text"]
    if len(tokenizer(text)["input_ids"]) <= 2048:
        # data["text"] = data["text"]
        all_result.append(data)
    # all_len.append(len(tokenizer(text)["input_ids"]))

7405it [00:17, 426.88it/s]


In [16]:
with open("data/teddst_dev.json", "w", encoding="utf-8") as f:
    json.dump(input_data, f, ensure_ascii=False, indent=4)

In [7]:
print(input_data[0]["label"])

**Answer**: yes
**Supporting Sentences**: [4] Scott Derrickson (born July 16, 1966) is an American director, screenwriter and producer.
[17] Edward Davis Wood Jr. (October 10, 1924 – December 10, 1978) was an American filmmaker, actor, writer, producer, and director.



In [16]:
count = len(list(filter(lambda x: x < 2048, all_len)))
print(count)

6539
